# Geocoding API


In [1]:
import herepy
import pandas as pd
from tqdm import tqdm 
import time
import math

## Tuguegarao

In [22]:
df = pd.read_csv('ror_relief_data.csv')

In [23]:
df['STATUS'].unique()

array(['Received Goods', 'Partially Received Goods', 'No Update',
       'Waiting for Goods'], dtype=object)

In [24]:
df = df[(df['STATUS'] != 'Received Goods')]

In [25]:
df = df[df['ADDRESS'] != '']

## Google Maps Geocoding

In [13]:
import googlemaps
from datetime import datetime

# PUT YOUR OWN API KEY
gmaps = googlemaps.Client(key='')

In [26]:
lat_list = []
lng_list = []
with tqdm(total=df.shape[0]) as pbar:
    for index, row in df.iterrows():
        pbar.update(1)
        
        geocode_result = gmaps.geocode(row['ADDRESS'])
        
        try:
            lat = geocode_result[0]['geometry']['location']['lat']
            long = geocode_result[0]['geometry']['location']['lng']
            
            lat_list.append(lat)
            lng_list.append(long)
        except:
            lat_list.append(0)
            lng_list.append(0)
        time.sleep(2)

100%|██████████| 175/175 [06:54<00:00,  2.37s/it]


In [27]:
df['lat'] = lat_list
df['lng'] = lng_list

In [28]:
df.to_csv('tuguegeocoded_locations.csv')

## GEOPANDAS

In [29]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

#### Converting our scraped data to geopandas format

In [63]:
df = pd.read_csv('tuguegeocoded_locations.csv')

In [64]:
df

,Unnamed: 0,NAME,CONTACT #,ADDRESS,TOWN/CITY,NEED/S,HEAD COUNT,VERIFIED BY,STATUS,URGENCY,NOTES,LAST TIME OF UPDATE,lat,lng
0,2,Ella Fernando,9652551955,"Alcala, Cagayan",Alcala,Mefenamic amoxicillin at biogesic po Sana para...,NaN,kiyoko-san via text,Partially Received Goods,NaN,Mefenamic amoxicillin at biogesic po Sana para...,SMS 8:42 PM 11/18/20,17.904972,121.651126
1,3,Gem Contillo,9208277257,baybayog alcala cagayan baybayog bridge near b...,Alcala,Need food and water,NaN,CBR & SMS by @jessy,No Update,NaN,"""Current situation: Need din nila sana ng nurs...",10:17 AM 11/19/20,17.892204,121.691849
2,4,Jenifer Preza Granada,9061801954,"Brgy. West Baculod, Alcala",Alcala,safe but need assistance and reliefs,NaN,last contacted by @kombucharm,No Update,NaN,NaN,10:18 AM 11/19/20,17.898744,121.671086
3,5,Luciana Duruin,9352321193,"Damurog, Alacala, Cagayan Valley",Alcala,"NEED: kumot, banig, biogesic, neozep, & bioflu",NaN,as per @kombucharm,Partially Received Goods,NaN,"they have food. NEED: kumot, banig, biogesic, ...",SMS 8:42 PM 11/18/20,17.878753,121.665383
4,8,April Vanne Leaño,9171647758,"Baculud Amulung Cagayan, Zone 4, Baculud Amulu...",Amulung,Need food and supplies,NaN,kiyoko-san,Waiting for Goods,NaN,"""wala pong natanggap na relief dto sa area namin""",10:19 AM 11/19/20,17.855909,121.733795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,261,Judy narag soriano,9615343629,"needs FOOD, WATER, & SABON",Linao,NaN,NaN,@jessy,No Update,NaN,NaN,1:45 am by @bea,0.000000,0.000000
171,262,Erlinda Mallari,9977134024,540 Zone 4 Dangarang St. Brgy Buntun,Tuguegarao,Need Supplies,NaN,@LeandroGumapos,Waiting for Goods,NaN,NaN,"8:00AM, 11/16/20 by @LeandroGumapos",17.612220,121.693403
172,263,Wilme Ramos,9954758593,NaN,NaN,Load,NaN,Please verify kung hindi scam,No Update,NaN,NaN,NaN,45.522208,-122.986328
173,264,Cesar G. Castillo,NaN,"Alcala, Centro Sur, Cagayan",Cagayan,need relief goods,NaN,@nyx discord,No Update,NaN,safe but need relief goods,4:41 PM | 11/18/20 by @nyx discord,17.895517,121.652552


In [53]:
df = df[(df['lng'] >= 120.68) & (df['lng'] <= 122.4)]
df = df[(df['lat'] >= 17.5) & (df['lat'] <= 19.6)]

In [65]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lng, df.lat))

# Mapping

In [66]:
import folium
import folium.plugins as plugins


In [67]:
df = df.reset_index(drop = True)

In [58]:
coo = []
info = []

for geo in df['geometry'].unique():
    coo.append(geo)
    info.append(df[df['geometry'] == geo][['NAME', 'CONTACT #','NEED/S','ADDRESS','HEAD COUNT', 'VERIFIED BY','NOTES']].to_html())
    
tables = pd.DataFrame({'geometry': coo, 'info': info})

In [68]:
tables = tables.reset_index(drop = True)
tables = gpd.GeoDataFrame(tables, geometry='geometry')


In [69]:
gdf = gdf.merge(tables, on='geometry').drop_duplicates().reset_index(drop = 'True')

In [74]:
# df['info'] = [' DETAILS: '.join(i) for i in zip(df['address'].astype(str),df['NOTES'].astype(str))]
# df['info'] = [' AREA: '.join(i) for i in zip(df['info'].astype(str),df['AREA'].astype(str))]
# df['info'] = [' CONTACT NUMBER: '.join(i) for i in zip(df['info'].astype(str),df["CONTACT NUMBER"].astype(str))]
# df['info'] = [' PERSONS TO RESCUE: '.join(i) for i in zip(df['info'].astype(str),df["PERSONS TO RESCUE"].astype(str))]
# df['info'] = [' DATE AND TIME CONTACTED: '.join(i) for i in zip(df['info'].astype(str),df["DATE AND TIME CONTACTED"].astype(str))]
# df['info'] = 'ADDRESS: ' + df['info']

In [45]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [70]:
gdf['test'] = "<div style='height:250px;overflow:auto;'>"+ gdf['info'] +"</div>" 

In [71]:
len(gdf)

128

In [74]:
map1 = folium.Map(
    location=[17.6245371,121.7259032 ],
    zoom_start=10,
)


loc = 'Tuguegarao Relief Request Mapping'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

map1.get_root().html.add_child(folium.Element(title_html))

loc = 'Last Updated: 2:22 PM 11-21-2020 <br> Mapped using Google Maps Geocoding.<br> Note that these are rough estimates of the coordinates <br> as identified by the Google API based from address description.<br> Data Source: <a href = "https://docs.google.com/spreadsheets/d/1cClZuGoPa5ZajqoV7dRS8bPNAk4iPL6KkkXPnSZ9ppE/edit?usp=sharing"> #RequestForRescue </a> <br> Generated by <a href = "https://magoanalytics.github.io/">Mago Analytics </a>'
title_html = '''
             <p align="center" style="font-size:9">{}</p>
             '''.format(loc)   

map1.get_root().html.add_child(folium.Element(title_html))

# Add custom basemaps
basemaps['Google Maps'].add_to(map1)

gdf.apply(lambda row:folium.CircleMarker(location=[row["lat"], row["lng"]], popup = row['test'],fill_color='red', height=100).add_to(map1), axis=1)
map1

In [75]:
map1.save('tuguegarao_relief_v1.html')

## --------------------------------------------